In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 34.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("Loading data")
print("x_train.shape: ", x_train.shape)

4422102/4422102 [==============================] - 0s 0us/step
Loading data
x_train.shape:  (60000, 28, 28)


In [5]:
#CNN requires 3 dimensions
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train.shape: ", x_train.shape)


x_train.shape:  (60000, 28, 28, 1)


In [6]:
# number of classes
K = len(set(y_train))
print("num classes: ", + K)

num classes:  10


In [7]:
%%time
# Input layer
print("Shape: ", x_train[0].shape)
i = Input(shape=x_train[0].shape)

# no. params = kernalSize + bias * numFilters
# conv2d param# 1904 = (5x5 + 1) * 112
x = Conv2D(112, (4, 4), padding='Same', activation=None)(i)
# batch_normalization
x = BatchNormalization()(x)
# activation
x = Activation('relu')(x)

# conv2d_1 param# 64,576 = (3*3*112 + 1) * 64
x = Conv2D(64, (3, 3), padding='Same', activation=None)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# conv2d_2 params# 204,928 = (5*5*64 + 1) * 128
x = Conv2D(128, (5, 5), padding='Same', activation=None)(x)
x = BatchNormalization()(x)
#activation_2 (outputSize: 28,28,128)
x = Activation('relu')(x)

# max_pooling2d (outputSize: 14,14,128)
x = MaxPooling2D((2, 2))(x)

# flatten (outputsize: 14*14*128 = 25,088)
x = Flatten()(x)

# dense (params: (inputSize + bias) * units = (25,088 + 1) * 208 = 5,218,512)
x = Dense(208, activation='relu')(x)
#dropout
x = Dropout(0.2)(x)
#dense_1
x = Dense(160, activation='relu')(x)
#dropout_1
x = Dropout(0.2)(x)
#dense_2
x = Dense(128, activation='relu')(x)
#dropout_2
x = Dropout(0.2)(x)
# dense_3
x = Dense(10, activation='softmax')(x)

model = Model(i, x)
model.summary()

Shape:  (28, 28, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 112)       1904      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 112)      448       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 112)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        64576     
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                         

In [16]:
%%time
import scipy.ndimage
#from scipy import ndimage
# compile and fit
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# image augmentation
batch_size=32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1)
train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
r = model.fit(train_generator, validation_data=(x_test, y_test), steps_per_epoch=steps_per_epoch, epochs=1)

ImportError: dlopen(/Users/julia/miniforge3/lib/python3.10/site-packages/scipy/special/_ufuncs.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <514D43F8-FF0E-3306-8D53-AE0888CD4953> /Users/julia/miniforge3/lib/python3.10/site-packages/scipy/special/_ufuncs.cpython-310-darwin.so
  Reason: tried: '/Users/julia/miniforge3/lib/python3.10/site-packages/scipy/special/liblapack.3.dylib' (no such file), '/Users/julia/miniforge3/lib/python3.10/site-packages/scipy/special/../../../../liblapack.3.dylib' (no such file), '/Users/julia/miniforge3/lib/python3.10/site-packages/scipy/special/liblapack.3.dylib' (no such file), '/Users/julia/miniforge3/lib/python3.10/site-packages/scipy/special/../../../../liblapack.3.dylib' (no such file), '/Users/julia/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/julia/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib

In [ ]:
%%time
print(r.history.keys())

# plot loss per iteration
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
# plot accuracy per iteration
%%time
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()